# **$K_p$**

# Installing third-party packages

In [120]:
#@title Installation of packages
%%capture
!pip install --prefer-binary pyscf
!pip install pyscf[geomopt]
! pip install ase chemicals pubchempy
! pip install --upgrade x3dase
! sudo apt install cm-super dvipng texlive-latex-extra texlive-latex-recommended

In [121]:
#@title Selecting the substance (by name) and retrieving the CID from PubChem https://pubchem.ncbi.nlm.nih.gov/
import pubchempy as pcp

molA = input('Name of your substance: ')
queryA0 = pcp.get_compounds(molA,'name')


queryA = pcp.get_compounds(molA,'name',record_type='3d')

if(len(queryA)==0): queryA = queryA0


cid_A = pcp.get_cids(molA)
if len(cid_A) == 0: print('Molecule not found')
else: print('CID of molecule: ',cid_A)

m = queryA[0]
nat_A = len(m.atoms)
f = open('dumpA.xyz','w')
if nat_A > 1:
  f.write(str(nat_A)+'\n\n')
  for i,a in enumerate(m.atoms):
    if a.z is None: a.z = 0.0
    f.write(m.elements[i]+' '+str(a.x)+' '+str(a.y)+' '+str(a.z)+'\n')
else:                            f.write(m.elements[0]+' 0. 0. 0.\n')
f.close()

Name of your substance: n2o4
CID of molecule:  [25352]


In [128]:
from pyscf import gto, scf
from pyscf.hessian import thermo
from pyscf.geomopt.geometric_solver import optimize

#number_of_unpaired_elecs = 1
spin = 0
mol = gto.M(
    atom = "dumpA.xyz",
    spin=spin,
    basis='3-21g')

#mol.spin = mol.nelec[0]-mol.nelec[1]

if spin == 0:
  mf = mol.RHF()
else:
  mf = mol.UHF()

# geometric
mol_eq = optimize(mf, maxsteps=100)
print(mol_eq.atom_coords())

if spin == 0:
  mf = mol_eq.RHF().run()
else:
  mf = mol_eq.UHF().run()

hessian = mf.Hessian().kernel()

freq_info = thermo.harmonic_analysis(mf.mol, hessian)
thermo_info = thermo.thermo(mf, freq_info['freq_au'], 298.15, 101325)
mass = mol.atom_mass_list(isotope_avg=True)
rot_info = thermo.rotation_const(mass,mol_eq.atom_coords())
symmetry_n = thermo.rotational_symmetry_number(mol_eq)

print(freq_info['freq_wavenumber'])
print(thermo_info)
print(rot_info)
print(symmetry_n)



geometric-optimize called with the following command line:
/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py -f /root/.local/share/jupyter/runtime/kernel-0a736671-b40b-4741-bd27-64a285b548a1.json

                                        ())))))))))))))))/                     
                                    ())))))))))))))))))))))))),                
                                *)))))))))))))))))))))))))))))))))             
                        #,    ()))))))))/                .)))))))))),          
                      #%%%%,  ())))))                        .))))))))*        
                      *%%%%%%,  ))              ..              ,))))))).      
                        *%%%%%%,         ***************/.        .)))))))     
                #%%/      (%%%%%%,    /*********************.       )))))))    
              .%%%%%%#      *%%%%%%,  *******/,     **********,      .))))))   
                .%%%%%%/      *%%%%%%,  **              ********    


Geometry optimization cycle 1
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -1.308500   1.103100   0.000500    0.000000  0.000000  0.000000
   O   1.308400   1.103100  -0.000500   -0.000000  0.000000  0.000000
   O  -1.308500  -1.103200  -0.000700    0.000000  0.000000  0.000000
   O   1.308500  -1.103100   0.000700    0.000000  0.000000  0.000000
   N  -0.749000   0.000000   0.000300    0.000000  0.000000 -0.000000
   N   0.749000   0.000000  -0.000300    0.000000  0.000000  0.000000
converged SCF energy = -405.645386365123
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0304074797     0.0115218416    -0.0000529795
1 O     0.0303769000     0.0114700355     0.0000530134
2 O    -0.0304298940    -0.0115774801    -0.0000765368
3 O     0.0303989717    -0.0114966875     0.0000764900
4 N     0.0847451039     0.0000634916     0.0001842320
5 N    -0.0846836019     0.0000187989

Step    0 : Gradient = 5.565e-02/8.475e-02 (rms/max) Energy = -405.6453863651
Hessian Eigenvalues: 2.30000e-02 3.04802e-02 3.04809e-02 ... 8.82791e-01 8.82793e-01 8.82973e-01



Geometry optimization cycle 2
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -1.196277   1.145311   0.000548    0.112223  0.042211  0.000048
   O   1.196182   1.145341  -0.000548   -0.112218  0.042241 -0.000048
   O  -1.196253  -1.145403  -0.000672    0.112247 -0.042203  0.000028
   O   1.196254  -1.145340   0.000672   -0.112246 -0.042240 -0.000028
   N  -0.760290  -0.000010   0.000202   -0.011290 -0.000010 -0.000098
   N   0.760284   0.000001  -0.000202    0.011284  0.000001  0.000098
converged SCF energy = -405.648124765668
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0160699340     0.0276711634    -0.0000206736
1 O    -0.0160925436     0.0276517216     0.0000206815
2 O     0.0160561919    -0.0277088454    -0.0000562260
3 O    -0.0160755333    -0.0276731500     0.0000562019
4 N     0.0290291633     0.0000442554     0.0001157927
5 N    -0.0289872124     0.0000148550

Step    1 : Displace = 9.813e-02/1.199e-01 (rms/max) Trust = 1.000e-01 (=) Grad = 3.104e-02/3.202e-02 (rms/max) E (change) = -405.6481247657 (-2.738e-03) Quality = 0.152
Hessian Eigenvalues: 2.30000e-02 3.04802e-02 3.04809e-02 ... 8.82792e-01 8.82918e-01 1.18974e+00


...


Geometry optimization cycle 6
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -1.281320   1.111385   0.000245   -0.001698  0.000482  0.000100
   O   1.281237   1.111444  -0.000245    0.001683  0.000498 -0.000100
   O  -1.281289  -1.111461  -0.000022   -0.001724 -0.000479  0.000292
   O   1.281285  -1.111431   0.000022    0.001709 -0.000495 -0.000292
   N  -0.818697  -0.000033  -0.000415   -0.001950 -0.000003 -0.000701
   N   0.818684  -0.000004   0.000415    0.001979 -0.000003  0.000701
converged SCF energy = -405.659404993319
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O     0.0000453876    -0.0000273211     0.0000773787
1 O    -0.0000530803    -0.0000188952    -0.0000773731
2 O     0.0000452936     0.0000302830     0.0000739745
3 O    -0.0000528060     0.0000213818    -0.0000739670
4 N    -0.0000213071    -0.0000029696    -0.0002381608
5 N     0.0000365123    -0.0000024790

Step    5 : Displace = 1.891e-03/2.095e-03 (rms/max) Trust = 1.388e-01 (+) Grad = 1.583e-04/2.409e-04 (rms/max) E (change) = -405.6594049933 (-6.385e-06) Quality = 0.980
Hessian Eigenvalues: 2.29939e-02 3.04805e-02 3.39739e-02 ... 8.82792e-01 8.82920e-01 1.19867e+00



Geometry optimization cycle 7
Cartesian coordinates (Angstrom)
 Atom        New coordinates             dX        dY        dZ
   O  -1.281539   1.111436  -0.000017   -0.000219  0.000050 -0.000261
   O   1.281489   1.111450   0.000017    0.000252  0.000006  0.000261
   O  -1.281499  -1.111514  -0.000184   -0.000210 -0.000052 -0.000162
   O   1.281528  -1.111439   0.000184    0.000243 -0.000008  0.000162
   N  -0.818902  -0.000031   0.000376   -0.000204  0.000002  0.000791
   N   0.818823  -0.000002  -0.000376    0.000139  0.000002 -0.000791
converged SCF energy = -405.65940503654
--------------- RHF_Scanner gradients ---------------
         x                y                z
0 O    -0.0000064427     0.0000523359    -0.0000676417
1 O     0.0000230028     0.0000336026     0.0000676229
2 O    -0.0000060884    -0.0000517274    -0.0000697221
3 O     0.0000227548    -0.0000332242     0.0000697006
4 N    -0.0000603373    -0.0000006055     0.0002171922
5 N     0.0000271108    -0.0000003813 

Step    6 : Displace = 5.348e-04/7.960e-04 (rms/max) Trust = 1.963e-01 (+) Grad = 1.451e-04/2.254e-04 (rms/max) E (change) = -405.6594050365 (-4.322e-08) Quality = 0.095
Hessian Eigenvalues: 2.29939e-02 3.04805e-02 3.39739e-02 ... 8.82792e-01 8.82920e-01 1.19867e+00
Converged! =D

    #==========================================================================#
    #| If this code has benefited your research, please support us by citing: |#
    #|                                                                        |#
    #| Wang, L.-P.; Song, C.C. (2016) "Geometry optimization made simple with |#
    #| translation and rotation coordinates", J. Chem, Phys. 144, 214108.     |#
    #| http://dx.doi.org/10.1063/1.4952956                                    |#
    #==========================================================================#
    Time elapsed since start of run_optimizer: 9.462 seconds


[[-2.42175781e+00  2.10030902e+00 -3.17002738e-05]
 [ 2.42166313e+00  2.10033704e+00  3.15590497e-05]
 [-2.42168297e+00 -2.10045665e+00 -3.47563219e-04]
 [ 2.42173657e+00 -2.10031440e+00  3.47399836e-04]
 [-1.54749960e+00 -5.92947526e-05  7.11387092e-04]
 [ 1.54735170e+00 -4.69653663e-06 -7.11082484e-04]]
converged SCF energy = -405.659405036571
[ 107.70170719  311.74289849  434.25711166  508.85890284  546.81380019
  760.19733065  793.98397689  873.60910819 1222.91918862 1282.04607322
 1523.44823799 1545.64630063]
{'temperature': (298.15, 'K'), 'pressure': (101325, 'Pa'), 'E0': (-405.6594050365713, 'Eh'), 'S_elec': (0.0, 'Eh/K'), 'Cv_elec': (0, 'Eh/K'), 'Cp_elec': (0, 'Eh/K'), 'E_elec': (-405.6594050365713, 'Eh'), 'H_elec': (-405.6594050365713, 'Eh'), 'S_trans': (6.289908387066597e-05, 'Eh/K'), 'Cv_trans': (4.750215777928532e-06, 'Eh/K'), 'Cp_trans': (7.917026296547553e-06, 'Eh/K'), 'E_trans': (0.0014162768341893916, 'Eh'), 'H_trans': (0.0023604613903156525, 'Eh'), 'rot_const': (array(

In [168]:
#@title Exact (determined from statistical mechanics) and approximate values of $\mu^0$
# Defining some constants
m_u    = 1.66053886e-27      # atomic mass constant in kg
h      = 6.6260693e-34       # Planck's constant in J*s
k_B    = 1.3806505e-23       # Boltzmann's constant in J/K
c      = 2.99792558e8        # Speed of light in m/s
bar2Pa = 1e5                 # 1 bar to N/m**2
Pa2bar = 1e-5                # 1 Pa=N/m**2 to bar
p0     = 1                   # standard pressure in bar
R      = 8.314472e-3         # Gas constant in kJ/(K mol)
H2kcal = 627.509             # Hartree to kcal/mol

import numpy as np

def G_m_0(temp,m,A,B,C,lin_A,nat_A,sigma,*freq):
  import math
  q_t = (2*math.pi*m*m_u*k_B*temp/h/h)**1.5*k_B*temp/bar2Pa
  St = R * np.log(q_t)
  Ut = 1.5 * R * temp
  q_r = 1 ; q_v = 1 ; Sr = 0 ; Sv = 0 ; Ur = 0 ; Uv = 0
  if nat_A > 1:
    rA  = h * c * 100 * A / k_B
    rB  = h * c * 100 * B / k_B
    rC  = h * c * 100 * C / k_B
    if lin_A == 0:
      q_r = 1 / sigma * math.sqrt(math.pi)*(temp**1.5/math.sqrt(rA*rB*rC))
      Sr = R * (1.5 + np.log(q_r))
      Ur = 1.5 * R * temp
    else:
      if rA == 0:
        rl = math.sqrt(rB*rC)
      elif rB == 0:
        rl = math.sqrt(rA*rC)
      else:
        rl = math.sqrt(rA*rB)
      q_r = 1 / sigma * temp / rl
      Sr = R * (1 + np.log(q_r))
      Ur = R * temp
    for ele in freq:
      rv   = h * c * 100 * float(ele) / k_B
      q_v *= 1 / (1 - np.exp(-rv/temp))
      Sv  += R * (rv / temp / (np.exp(rv/temp) - 1) - np.log(1 - np.exp(-rv/temp)) )
      Uv += R * rv / (np.exp(rv/temp) - 1)
  q = q_t * q_r * q_v
  return -R * temp * np.log(q_t), -R * temp * np.log(q_r), -R * temp * np.log(q_v), - R * temp * np.log(q), q_v


In [169]:
tot_mass=sum(mass)
A = rot_info[0] * 0.033357
B = rot_info[1] * 0.033357
C = rot_info[2] * 0.033357
if A == B and C <= 0.001: lin = 1
else: lin=0
sigma = symmetry_n
freq = freq_info['freq_wavenumber']
g_t, g_r, g_v, g , q_v= G_m_0(298.15,tot_mass,A,B,C,lin,nat_A,sigma,*freq)
print(g_t/H2kcal/4.184, g_r/H2kcal/4.184, g_v/H2kcal/4.184, g/H2kcal/4.184)

print('q_v',q_v)



e0 = thermo_info['E0'][0]
zpe = thermo_info['ZPE'][0]

print('cojones',thermo_info['G_trans'][0])
print('cojones',thermo_info['G_rot'][0])
print('cojones',thermo_info['G_vib'][0])


print(e0,zpe,e0+zpe,e0+zpe+g/H2kcal/4.184)

-0.016405368619716525 -0.010940339706677507 -0.0014333261176315748 -0.028779034444025606
q_v 4.563335341617352
cojones -0.016392900465723406
cojones -0.010940338841307408
cojones 0.021146110706456292
-405.6594050365713 0.02257943113101926 -405.63682560544027 -405.6656046398843
